In [1]:
import torch
from PIL import Image
from torchvision import transforms
import cv2
import numpy as np

In [2]:
# Загрузка модели
ckpt = torch.load("fire_l.pt")
torch.save(ckpt, "updated-fire_l.pt")
model_path = 'updated-fire_l.pt'
checkpoint = torch.load(model_path)
model = checkpoint['model']

WARNING  'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING  'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



In [3]:
# Функция для сжатия изображения до подходящего размера
def resize_image(image, target_width, target_height):
    return cv2.resize(image, (target_width, target_height))

In [4]:
# Функция для изменения координат обратно к исходному размеру изображения
def scale_coordinates(x, y, width, height, original_width, original_height):
    x_original = int(x * original_width / target_width)
    y_original = int(y * original_height / target_height)
    width_original = int(width * original_width / target_width)
    height_original = int(height * original_height / target_height)
    return x_original, y_original, width_original, height_original

In [5]:
# Загрузка изображения
# image_path = "forest_fire/Testing/fire/abc186.jpg"
image_path = "photo_433965.jpeg"
image = cv2.imread(image_path)

# Проверка, что изображение загружено успешно
if image is None:
    print("Ошибка загрузки изображения.")
    exit()

In [6]:
# Сохранение размеров исходного изображения
original_height, original_width = image.shape[:2]

In [7]:
# Задание новых размеров для сжатия изображения
target_width, target_height = 224, 224

In [8]:
# Сжатие изображения до подходящего размера
resized_image = resize_image(image, target_width, target_height)

In [9]:
# Преобразование изображения в тензор
transform = transforms.Compose([transforms.ToTensor()])
image_tensor = transform(resized_image).unsqueeze(0)  # добавляем размерность пакета

In [10]:
# Преобразование типа входных данных к типу, поддерживаемому моделью
image_tensor = image_tensor.float()

In [11]:
# Преобразование весов модели к типу данных FloatTensor
model.float()

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(320, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
    

In [12]:
# Получение предсказаний от модели
with torch.no_grad():
    predictions = model(image_tensor)
    # predictions = model(image)

In [13]:
def draw_boxes(image, predictions):
    # Копируем изображение, чтобы не изменять оригинал
    image_with_boxes = image.copy()

    # Переменные для проверки наличия огня или дыма
    fire_detected = False
    smoke_detected = False

    # Проход по всем обнаруженным объектам
    for pred in predictions[0][0]:
        # Извлечение координат прямоугольника и уверенности
        x_center, y_center, width, height = pred[:4]

        # Преобразование координат в координаты верхнего левого угла и размеры
        x1 = int((x_center - width / 2) * image.shape[1])
        y1 = int((y_center - height / 2) * image.shape[0])
        x2 = int((x_center + width / 2) * image.shape[1])
        y2 = int((y_center + height / 2) * image.shape[0])

        # Определение цвета прямоугольника в зависимости от типа объекта
        color = (255, 0, 0) if pred[5] == 0 else (192, 192, 192)  # Красный для огня, серый для дыма

        # Рисуем прямоугольник на изображении
        cv2.rectangle(image_with_boxes, (x1, y1), (x2, y2), color, 2)

        # Определение текста для подписи в зависимости от типа объекта
        label = "Огонь" if pred[5] == 0 else "Дым"

        # Добавляем подпись к прямоугольнику
        cv2.putText(image_with_boxes, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Проверяем, обнаружен ли огонь или дым
        if pred[5] == False:
            fire_detected = True
        else:
            smoke_detected = True

    # Показываем изображение с нарисованными прямоугольниками и подписями
    cv2.imshow("Image with Boxes", image_with_boxes)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Проверяем, был ли обнаружен огонь и/или дым и выводим соответствующее сообщение
    if fire_detected and smoke_detected:
        print("На изображении обнаружены огонь и дым.")
    if fire_detected:
        print("На изображении обнаружен огонь.")
    if smoke_detected:
        print("На изображении обнаружен дым.")
    else:
        print("На изображении не обнаружено огня и дыма.")

In [14]:
# def draw_boxes(image, predictions):
#     # Копируем изображение, чтобы не изменять оригинал
#     image_with_boxes = image.copy()

#     # Проходим по всем предсказаниям
#     for pred in predictions[0][0]:
#         # Извлекаем координаты и уверенность прямоугольника из предсказания
#         x_center, y_center, width, height, confidence = pred.tolist()

#         # Преобразуем координаты в координаты верхнего левого угла
#         x1 = int((x_center - width / 2) * image.shape[1])
#         y1 = int((y_center - height / 2) * image.shape[0])
#         x2 = int((x_center + width / 2) * image.shape[1])
#         y2 = int((y_center + height / 2) * image.shape[0])

#         # Рисуем прямоугольник на изображении
#         cv2.rectangle(image_with_boxes, (x1, y1), (x2, y2), (0, 255, 0), 2)

#         # Добавляем текст с уверенностью
#         text = f"Confidence: {confidence:.2f}"
#         cv2.putText(image_with_boxes, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

#     # Показываем изображение с нарисованными прямоугольниками
#     cv2.imshow("Image with Boxes", image_with_boxes)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()


In [15]:
# Вызов функции для рисования прямоугольников на изображении с предсказаниями
draw_boxes(image, predictions)

На изображении обнаружен дым.
